# Window Functions

There are three types of indexing functions in DAX: index, offset, AND WINDOW functions.

Notes: 

1. TOPN covers some features of index. Ex: returning first row. But, index has added advantage of picking any rows: example 3rd row.

2. Of the three window functions, only Index has the capability to sort a table by columns when the table was created on the fly.

3. Index always returns unique values while TOPN can return two different rows with identical value.

4. ORDER BY in INDEX is one word like this: ORDERBY

## Index

> Show 2nd highest sales amount using DAX studio

    EVALUATE
        VAR BrandsAndSales =
            ADDCOLUMNS (
                VALUES ( DimProduct[BrandName] ),
                "@Sales", [Sales_Amount]
            )
        RETURN
            INDEX (
                2,                              -- use -1 for the last row or brand with the least sales amount
                BrandsAndSales,
                ORDERBY ( [@Sales], DESC )
            )

> Using Power BI New Table

    2nd highest sales = 

    VAR BrandsAndSales = 
                ADDCOLUMNS (
                    VALUES ( DimProduct[BrandName] ),
                    "@Sales", [Sales_Amount]
                )
    RETURN    
        INDEX (
            2,
            BrandsAndSales,
            ORDERBY ( [@Sales], DESC )
            )

# 2nd most highest sales for every product colour

DAX STUDIO

    EVALUATE
        VAR BrandsAndSales =
            ADDCOLUMNS (
                ALL ( DimProduct[ColorName], DimProduct[BrandName] ),
                "@Sales", ROUND([Sales_Amount],0)
            )
        RETURN
        
        	INDEX(
		    2,
		    BrandsAndSales, 
		    ORDERBY ([@Sales], DESC),
		    PARTITIONBY(DimProduct[ColorName])
        	)

## Sales by Product Colour and Brand

    EVALUATE
        VAR BrandsAndSales =
            ADDCOLUMNS (
                ALL ( DimProduct[ColorName], DimProduct[BrandName] ),
                "@Sales", ROUND([Sales_Amount],0)
            )
        RETURN
        
        	BrandsAndSales ORDER BY DimProduct[ColorName], [@Sales] DESC

# Sales by Product Colour and Brand vs top sales amount

> Note that table is optional when using index as in below since it will by default take DimProduct 
      
	  EVALUATE
            ADDCOLUMNS (
                ALL ( DimProduct[ColorName], DimProduct[BrandName] ),
                "@Sales", ROUND([Sales_Amount],0),
                "Top Sales",
                	CALCULATE(
                		ROUND([Sales_Amount],0),
                		INDEX(
                			1,
                			ORDERBY (DimProduct[BrandName]) 
                		)
                	)
            )

# Window to Calculate Cumulative with and without partition

### Without Partition
    EVALUATE
    SUMMARIZECOLUMNS (
        'Date'[Year Month Number],
        'Date'[Year Month],
        "Sales Amount", [Sales Amount],
        "Sales RT",
            SUMX (
                WINDOW (
                    1, ABS,                                  -- Current row
                    0, REL,                                  -- Previous row
                    ORDERBY (
                        'Date'[Year Month Number], ASC,
                        'Date'[Year Month], ASC
                    )
                ),
                [Sales Amount]
            )
    )
    ORDER BY 'Date'[Year Month Number]

|Date[Year Month Number]	|Date[Year Month]|	Sales Amount|	Sales RT|
|---------------------------|----------------|--------------|-----------|
|24209	                    |May 2017	|168,392.56|	168,392.56|
|24210	                    |June 2017	|263,600.69|	431,993.25|
|24211	                    |July 2017	|204,281.19|	636,274.44|
|24212	                    |August 2017	|312,793.50|	949,067.94|
|24213	                    |September 2017	|334,423.50|	1,283,491.44|
|24214	                    |October 2017	|402,067.05|	1,685,558.49|
|24215	                    |November 2017	|438,804.70|	2,124,363.19|
|24216	                    |December 2017	|908,941.83|	3,033,305.02|
|24217	                    |January 2018	|636,983.88|	3,670,288.90|
|24218	                    |February 2018	|788,062.88|	4,458,351.78|
|24219	                    |March 2018	|269,320.40|	4,727,672.19|

### with Partition

    EVALUATE
    SUMMARIZECOLUMNS (
        'Date'[Year Month Number],
        'Date'[Year Month],
        "Sales Amount", [Sales Amount],
        "Sales RT",
            SUMX (
                WINDOW (
                    1, ABS,
                    0, REL,
                    ORDERBY (
                        'Date'[Year Month Number], ASC,
                        'Date'[Year Month], ASC
                    ),
                    PARTITIONBY ( 'Date'[Year] )
                ),
                [Sales Amount]
            )
    )
    ORDER BY 'Date'[Year Month Number]

|Date[Year Month Number]	|Date[Year Month]	|Sales Amount	|Sales RT|
|---------------------------|-------------------|---------------|--------|
|24209	|May 2017	|168,392.56	|168,392.56|
|24210	|June 2017	|263,600.69	|431,993.25|
|24211	|July 2017	|204,281.19	|636,274.44|
|24212	|August 2017	|312,793.50	|949,067.94|
|24213	|September 2017	|334,423.50	|1,283,491.44|
|24214	|October 2017	|402,067.05	|1,685,558.49|
|24215	|November 2017	|438,804.70	|2,124,363.19|
|24216	|December 2017	|908,941.83	|3,033,305.02|
|24217	|January 2018	|636,983.88	|636,983.88|
|24218	|February 2018	|788,062.88	|1,425,046.76|
|24219	|March 2018	|269,320.40	|1,694,367.17|